In [109]:
import json
import requests
from datetime import datetime, timedelta
import nltk
import pandas as pd  
import requests
import time
from datetime import datetime
import yfinance as yf
import numpy as np
import joblib
import tensorflow as tf

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


2024-12-01 22:01:25.747451: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733072485.790729    2351 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733072485.802723    2351 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 22:01:25.891730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/aunno/nltk_data...
[nltk_data]   Package vader_lexicon

In [18]:
STOCKS = [
    'nvidia', 'tesla', 'apple', 'facebook', 'microsoft', 'amazon', 'amd',
    'broadcom', 'palantir', 'microstrategy', 'exxon', 'micron', 'jpmorgan', 'google',
    'salesforce', 'lilly', 'vistra', 'home depot', 'netflix', 'unitedhealth',
    'bank of america', 'costco', 'super micro', 'chevron', 'visa', 'coinbase'
]

TICKERS = [
    'NVDA', 'TSLA', 'AAPL', 'META', 'MSFT', 'AMZN', 'AMD',
    'AVGO', 'PLTR', 'MSTR', 'XOM', 'MU', 'JPM', 'GOOG',
    'CRM', 'LLY', 'VST', 'HD', 'NFLX', 'UNH', 'BAC',
    'COST', 'SMCI', 'CVX', 'V', 'COIN'
]

SUBREDDITS = ['Investing', 'Stocks', 'WallStreetBets', 'Options', 'GlobalMarkets']



In [54]:


def fetch_reddit_posts(subreddit, stocks, max_posts_per_stock=None):
    headers = {"User-Agent": "Mozilla/5.0"}
    stock_posts = {stock: [] for stock in stocks}
    base_url = f"https://reddit.com/r/{subreddit}/search.json"
    
    # Calculate the timestamp for 20 days ago (from now)
    twenty_days_ago = datetime.utcnow() - timedelta(days=20)
    twenty_days_ago_timestamp = int(twenty_days_ago.timestamp())
    
    for stock in stocks:
        after = twenty_days_ago_timestamp 
        
        while max_posts_per_stock is None or len(stock_posts[stock]) < max_posts_per_stock:
            params = {
                'q': stock,          # Search for the stock keyword
                'sort': 'new',       # Sort by the newest posts
                'limit': 10000,        # Maximum allowed posts per request
                'restrict_sr': True, # Restrict to the subreddit
                'after': after       
            }
            
            response = requests.get(base_url, headers=headers, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} while fetching {stock} from {subreddit}")
                break
            
            try:
                data = response.json()
            except ValueError as e:
                print("Error parsing JSON:", e)
                break
            
            if 'data' not in data or 'children' not in data['data']:
                break
            
            posts = data['data']['children']
            if not posts:  # Exit if no more posts
                break
            
            for post in posts:
                post_data = post['data']
                title = post_data['title'].lower()
                author = post_data.get('author', 'N/A')
                created_utc = post_data.get('created_utc')
                created_time = datetime.utcfromtimestamp(created_utc).strftime('%Y-%m-%d %H:%M:%S') if created_utc else "N/A"

                
                if created_utc >= twenty_days_ago_timestamp:
                    stock_posts[stock].append({
                        'title': title,
                        'author': author,
                        'created_time': created_time,
                        'selftext': post_data['selftext'].lower() if 'selftext' in post_data else '',
                    })
                
              
                if max_posts_per_stock and len(stock_posts[stock]) >= max_posts_per_stock:
                    break
            
            # Update `after` for pagination
            after = posts[-1]['data']['created_utc']
            if after < twenty_days_ago_timestamp: 
                break
    
    return stock_posts

In [55]:



# Function to perform sentiment analysis on text
def analyze_sentiment(text):
    sid = SentimentIntensityAnalyzer()
    result = sid.polarity_scores(text)
    return result

In [56]:

# Main function to fetch posts, analyze sentiment, and save to a DataFrame
def get_aggregated_stock_posts(subreddits, stocks, max_posts_per_stock=None):
    all_posts = []  # List to store all the post data for creating the DataFrame
    
    for subreddit in subreddits:
        print(f"Fetching posts from r/{subreddit}")
        stock_posts = fetch_reddit_posts(subreddit, stocks, max_posts_per_stock)
        
        for i, stock in enumerate(stocks):
            print(f"Processing stock: {TICKERS[i]}")
            
            for post in stock_posts[stock]:
                sentiment = analyze_sentiment(post['title']+ '. ' + post['selftext'])
                
                # Append the post data to the all_posts list
                all_posts.append({
                    'stock': stock,
                    'ticker': TICKERS[i],
                    'author': post['author'],
                    'created_time': post['created_time'],
                    'compound': sentiment['compound'],
                    'neg': sentiment['neg'],
                    'neu': sentiment['neu'],
                    'pos': sentiment['pos'],
                    'title': post['title'],
                    'selftext': post['selftext'],
                })
    
    # Create a DataFrame from the all_posts list
    df = pd.DataFrame(all_posts)
    
    return df


In [57]:



# Run the function and display DataFrame in Jupyter Notebook
df = get_aggregated_stock_posts(SUBREDDITS, STOCKS, max_posts_per_stock=1000)
df.head()  # Display the first few rows of the DataFrame

Fetching posts from r/Investing


/tmp/ipykernel_2351/3043965048.py:7: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  twenty_days_ago = datetime.utcnow() - timedelta(days=20)
/tmp/ipykernel_2351/3043965048.py:45: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  created_time = datetime.utcfromtimestamp(created_utc).strftime('%Y-%m-%d %H:%M:%S') if created_utc else "N/A"


Processing stock: NVDA
Processing stock: TSLA
Processing stock: AAPL
Processing stock: META
Processing stock: MSFT
Processing stock: AMZN
Processing stock: AMD
Processing stock: AVGO
Processing stock: PLTR
Processing stock: MSTR
Processing stock: XOM
Processing stock: MU
Processing stock: JPM
Processing stock: GOOG
Processing stock: CRM
Processing stock: LLY
Processing stock: VST
Processing stock: HD
Processing stock: NFLX
Processing stock: UNH
Processing stock: BAC
Processing stock: COST
Processing stock: SMCI
Processing stock: CVX
Processing stock: V
Processing stock: COIN
Fetching posts from r/Stocks
Processing stock: NVDA
Processing stock: TSLA
Processing stock: AAPL
Processing stock: META
Processing stock: MSFT
Processing stock: AMZN
Processing stock: AMD
Processing stock: AVGO
Processing stock: PLTR
Processing stock: MSTR
Processing stock: XOM
Processing stock: MU
Processing stock: JPM
Processing stock: GOOG
Processing stock: CRM
Processing stock: LLY
Processing stock: VST
Proces

,stock,ticker,author,created_time,compound,neg,neu,pos,title,selftext
0,nvidia,NVDA,luksev11,2024-11-30 15:12:16,0.9329,0.000,0.800,0.200,what’s the best investment you ever made?,"\n\nlike the title said, what was the best inv..."
1,nvidia,NVDA,PrestigiousCat969,2024-11-29 14:49:02,0.9524,0.000,0.781,0.219,s&amp;p 500 index gains for 2024 ranks among t...,2024 has proven unexpectedly good with the s&a...
2,nvidia,NVDA,HearAPianoFall,2024-11-28 23:44:37,0.9938,0.021,0.871,0.108,talking with my dad about nvidia over thanksgi...,"my dad is an active investor, has been for dec..."
3,nvidia,NVDA,Katamari_Demacia,2024-11-28 16:17:09,-0.3920,0.091,0.831,0.078,advice needed. should i pull off the market te...,"i’m 38, living in the u.s., and looking for so..."
4,nvidia,NVDA,Study-Sharp,2024-11-27 19:24:46,-0.4153,0.032,0.968,0.000,dca strat for individual stocks - $500 month,"\nhi everyone, \n\ni've these are my current s..."


In [61]:
pd.to_pickle(df,'temppost.pkl')

In [67]:
postdf=pd.read_pickle("temppost.pkl")
postdf

,stock,ticker,author,created_time,compound,neg,neu,pos,title,selftext
0,nvidia,NVDA,luksev11,2024-11-30 15:12:16,0.9329,0.000,0.800,0.200,what’s the best investment you ever made?,"\n\nlike the title said, what was the best inv..."
1,nvidia,NVDA,PrestigiousCat969,2024-11-29 14:49:02,0.9524,0.000,0.781,0.219,s&amp;p 500 index gains for 2024 ranks among t...,2024 has proven unexpectedly good with the s&a...
2,nvidia,NVDA,HearAPianoFall,2024-11-28 23:44:37,0.9938,0.021,0.871,0.108,talking with my dad about nvidia over thanksgi...,"my dad is an active investor, has been for dec..."
3,nvidia,NVDA,Katamari_Demacia,2024-11-28 16:17:09,-0.3920,0.091,0.831,0.078,advice needed. should i pull off the market te...,"i’m 38, living in the u.s., and looking for so..."
4,nvidia,NVDA,Study-Sharp,2024-11-27 19:24:46,-0.4153,0.032,0.968,0.000,dca strat for individual stocks - $500 month,"\nhi everyone, \n\ni've these are my current s..."
...,...,...,...,...,...,...,...,...,...,...
583,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
584,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
585,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
586,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,


In [68]:
postdf.sort_values(by='created_time',ascending=False)

,stock,ticker,author,created_time,compound,neg,neu,pos,title,selftext
13,tesla,TSLA,SevenDeMagnus,2024-12-01 10:04:38,0.9969,0.009,0.835,0.156,a second or third revisit of why warren buffet...,"hi investment friends, starting with the point..."
14,tesla,TSLA,HappyBend9701,2024-12-01 08:54:01,0.9848,0.025,0.801,0.174,thoughts on tesla in december,so right after tesla recently reached it's ath...
102,tesla,TSLA,idea_max_7777,2024-12-01 08:02:51,0.3025,0.098,0.822,0.080,difference between splitting large companies b...,not sure is this is just obvious and a stupid ...
130,microsoft,MSFT,idea_max_7777,2024-12-01 08:02:51,0.3025,0.098,0.822,0.080,difference between splitting large companies b...,not sure is this is just obvious and a stupid ...
157,google,GOOG,idea_max_7777,2024-12-01 08:02:51,0.3025,0.098,0.822,0.080,difference between splitting large companies b...,not sure is this is just obvious and a stupid ...
...,...,...,...,...,...,...,...,...,...,...
32,microsoft,MSFT,thedude510189,2024-11-11 14:49:33,0.9563,0.000,0.816,0.184,nuclear option in the face of ai?,"kind of a halfbaked thought process, but bear ..."
310,tesla,TSLA,xtreem_neo,2024-11-11 14:13:30,-0.3182,0.204,0.796,0.000,hedge funds shorting tesla just lost more than...,
12,nvidia,NVDA,Cold_Statement_73,2024-11-11 14:05:29,0.6301,0.040,0.846,0.114,investing advice or improvement,i am a uni student with no current income at t...
86,coinbase,COIN,Cold_Statement_73,2024-11-11 14:05:29,0.6301,0.040,0.846,0.114,investing advice or improvement,i am a uni student with no current income at t...


In [14]:
price_data=yf.download(TICKERS,period='1mo',group_by="column",auto_adjust=False)
        

[*********************100%***********************]  26 of 26 completed


In [13]:
df_list=[]

In [14]:
for ticker in TICKERS:
    print(f"Fetching data for {ticker}")
    stock_data= yf.download(ticker,period='1mo',interval='1d')
    stock_data['Stock']=ticker
    stock_data=stock_data.reset_index().reset_index()
    df_list.append(stock_data[['Date','Stock','Open','High','Close','Volume']])

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['NVDA']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Fetching data for NVDA
Fetching data for TSLA


[*********************100%***********************]  1 of 1 completed


Fetching data for AAPL


[*********************100%***********************]  1 of 1 completed


...

[*********************100%***********************]  1 of 1 completed


Fetching data for V


[*********************100%***********************]  1 of 1 completed


Fetching data for COIN


[*********************100%***********************]  1 of 1 completed


In [15]:
price_df=pd.concat(df_list,ignore_index=True)
price_df

Price,Date,Stock,Open,High,Close,Volume,Open,High,Close,Volume,...,Close,Volume,Open,High,Close,Volume,Open,High,Close,Volume
Ticker,,,NVDA,NVDA,NVDA,NVDA,TSLA,TSLA,TSLA,TSLA,...,CVX,CVX,V,V,V,V,COIN,COIN,COIN,COIN
0,2024-10-30,TSLA,NaN,NaN,NaN,NaN,258.040009,263.350006,257.549988,53993600.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-31,TSLA,NaN,NaN,NaN,NaN,257.989990,259.750000,249.850006,66575300.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-01,TSLA,NaN,NaN,NaN,NaN,252.039993,254.000000,248.979996,57544800.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-11-04,TSLA,NaN,NaN,NaN,NaN,244.559998,248.899994,242.839996,68802400.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-05,TSLA,NaN,NaN,NaN,NaN,247.339996,255.279999,251.440002,69282500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,2024-11-22,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,294.890015,311.369995,304.640015,12187200.0
546,2024-11-25,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,309.000000,317.459991,312.220001,14645900.0
547,2024-11-26,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,298.260010,309.200012,293.290009,13215700.0


In [12]:
price_df

Price,Date,Stock,Open,High,Close,Volume,Open,High,Close,Volume,...,Close,Volume,Open,High,Close,Volume,Open,High,Close,Volume
Ticker,,,NVDA,NVDA,NVDA,NVDA,TSLA,TSLA,TSLA,TSLA,...,CVX,CVX,V,V,V,V,COIN,COIN,COIN,COIN
0,2024-10-30,NVDA,139.539993,140.330002,139.339996,179418100.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-10-31,NVDA,137.600006,137.610001,132.759995,270039600.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-11-01,NVDA,134.699997,137.309998,135.399994,207127800.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-11-04,NVDA,137.210007,138.960007,136.050003,187528200.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-11-05,NVDA,137.449997,140.369995,139.910004,160537400.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
567,2024-11-22,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,294.890015,311.369995,304.640015,12187200.0
568,2024-11-25,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,309.000000,317.459991,312.220001,14645900.0
569,2024-11-26,COIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,298.260010,309.200012,293.290009,13215700.0


In [83]:
test= yf.download('AAPL',period='1mo',interval='1d')

[*********************100%***********************]  1 of 1 completed


In [84]:
test.columns=test.columns.droplevel(1)
test

Price,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,
2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900
2024-10-31,225.661728,225.910004,229.830002,225.369995,229.339996,64370100
2024-11-01,222.665024,222.910004,225.350006,220.270004,220.970001,65276700
2024-11-04,221.766006,222.009995,222.789993,219.710007,220.990005,44944500
2024-11-05,223.204422,223.449997,223.949997,221.139999,221.800003,28111300
2024-11-06,222.475235,222.720001,226.070007,221.190002,222.610001,54561100
2024-11-07,227.229996,227.479996,227.880005,224.570007,224.630005,42137700
2024-11-08,226.960007,226.960007,228.660004,226.410004,227.169998,38328800
2024-11-11,224.229996,224.229996,225.699997,221.500000,225.000000,42005600


In [85]:
test=test.reset_index()

In [86]:
test.columns.name=None
test

,Date,Adj Close,Close,High,Low,Open,Volume
0,2024-10-30,229.847122,230.100006,233.470001,229.550003,232.610001,47070900
1,2024-10-31,225.661728,225.910004,229.830002,225.369995,229.339996,64370100
2,2024-11-01,222.665024,222.910004,225.350006,220.270004,220.970001,65276700
3,2024-11-04,221.766006,222.009995,222.789993,219.710007,220.990005,44944500
4,2024-11-05,223.204422,223.449997,223.949997,221.139999,221.800003,28111300
5,2024-11-06,222.475235,222.720001,226.070007,221.190002,222.610001,54561100
6,2024-11-07,227.229996,227.479996,227.880005,224.570007,224.630005,42137700
7,2024-11-08,226.960007,226.960007,228.660004,226.410004,227.169998,38328800
8,2024-11-11,224.229996,224.229996,225.699997,221.500000,225.000000,42005600
9,2024-11-12,224.229996,224.229996,225.589996,223.360001,224.550003,40398300


In [87]:
test.drop(columns=['Adj Close'],inplace=True)
test

,Date,Close,High,Low,Open,Volume
0,2024-10-30,230.100006,233.470001,229.550003,232.610001,47070900
1,2024-10-31,225.910004,229.830002,225.369995,229.339996,64370100
2,2024-11-01,222.910004,225.350006,220.270004,220.970001,65276700
3,2024-11-04,222.009995,222.789993,219.710007,220.990005,44944500
4,2024-11-05,223.449997,223.949997,221.139999,221.800003,28111300
5,2024-11-06,222.720001,226.070007,221.190002,222.610001,54561100
6,2024-11-07,227.479996,227.880005,224.570007,224.630005,42137700
7,2024-11-08,226.960007,228.660004,226.410004,227.169998,38328800
8,2024-11-11,224.229996,225.699997,221.500000,225.000000,42005600
9,2024-11-12,224.229996,225.589996,223.360001,224.550003,40398300


In [95]:
price20days=test.tail(21)
price20days

,Date,Close,High,Low,Open,Volume
1,2024-10-31,225.910004,229.830002,225.369995,229.339996,64370100
2,2024-11-01,222.910004,225.350006,220.270004,220.970001,65276700
3,2024-11-04,222.009995,222.789993,219.710007,220.990005,44944500
4,2024-11-05,223.449997,223.949997,221.139999,221.800003,28111300
5,2024-11-06,222.720001,226.070007,221.190002,222.610001,54561100
6,2024-11-07,227.479996,227.880005,224.570007,224.630005,42137700
7,2024-11-08,226.960007,228.660004,226.410004,227.169998,38328800
8,2024-11-11,224.229996,225.699997,221.500000,225.000000,42005600
9,2024-11-12,224.229996,225.589996,223.360001,224.550003,40398300
10,2024-11-13,225.119995,226.649994,222.759995,224.009995,48566200


In [96]:
price20days['Date']=pd.to_datetime(price20days['Date']).dt.date

/tmp/ipykernel_2351/400943653.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price20days['Date']=pd.to_datetime(price20days['Date']).dt.date


In [69]:
postdf

,stock,ticker,author,created_time,compound,neg,neu,pos,title,selftext
0,nvidia,NVDA,luksev11,2024-11-30 15:12:16,0.9329,0.000,0.800,0.200,what’s the best investment you ever made?,"\n\nlike the title said, what was the best inv..."
1,nvidia,NVDA,PrestigiousCat969,2024-11-29 14:49:02,0.9524,0.000,0.781,0.219,s&amp;p 500 index gains for 2024 ranks among t...,2024 has proven unexpectedly good with the s&a...
2,nvidia,NVDA,HearAPianoFall,2024-11-28 23:44:37,0.9938,0.021,0.871,0.108,talking with my dad about nvidia over thanksgi...,"my dad is an active investor, has been for dec..."
3,nvidia,NVDA,Katamari_Demacia,2024-11-28 16:17:09,-0.3920,0.091,0.831,0.078,advice needed. should i pull off the market te...,"i’m 38, living in the u.s., and looking for so..."
4,nvidia,NVDA,Study-Sharp,2024-11-27 19:24:46,-0.4153,0.032,0.968,0.000,dca strat for individual stocks - $500 month,"\nhi everyone, \n\ni've these are my current s..."
...,...,...,...,...,...,...,...,...,...,...
583,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
584,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
585,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,
586,nvidia,NVDA,DueDiligence-Bot,2024-11-20 21:25:44,0.0000,0.000,1.000,0.000,nvidia surpasses market expectations across ke...,


In [70]:
#making date proper
postdf=postdf.rename(columns={'created_time': 'created_at'})
postdf=postdf.sort_values(by=['ticker','created_at'])
postdf=postdf.drop(columns=['author','compound','stock','title','selftext'])
postdf

,ticker,created_at,neg,neu,pos
126,AAPL,2024-11-11 15:53:47,0.033,0.809,0.158
125,AAPL,2024-11-11 19:21:20,0.020,0.897,0.082
29,AAPL,2024-11-12 14:12:27,0.144,0.738,0.118
325,AAPL,2024-11-12 17:27:52,0.047,0.843,0.110
324,AAPL,2024-11-12 21:10:40,0.000,1.000,0.000
...,...,...,...,...,...
81,V,2024-11-29 07:51:36,0.018,0.776,0.206
80,V,2024-11-30 20:17:32,0.087,0.807,0.106
444,XOM,2024-11-14 16:42:34,0.000,0.912,0.088
54,XOM,2024-11-24 18:36:08,0.046,0.744,0.210


In [71]:
# Filter posts related to Apple (AAPL) and format the 'created_at' column
postdf = postdf[postdf['ticker'].str.lower() == 'aapl'].copy()
postdf['Date'] = pd.to_datetime(postdf['created_at']).dt.date
postdf.drop(columns=['created_at','ticker'], inplace=True)

postdf

,neg,neu,pos,Date
126,0.033,0.809,0.158,2024-11-11
125,0.020,0.897,0.082,2024-11-11
29,0.144,0.738,0.118,2024-11-12
325,0.047,0.843,0.110,2024-11-12
324,0.000,1.000,0.000,2024-11-12
124,0.076,0.875,0.049,2024-11-13
532,0.045,0.765,0.190,2024-11-13
323,0.125,0.783,0.091,2024-11-13
123,0.181,0.712,0.107,2024-11-13
28,0.023,0.907,0.070,2024-11-14


In [72]:

aggregated_posts_df = postdf.groupby("Date", as_index=False)[["pos", "neg", "neu"]].mean()
aggregated_posts_df

,Date,pos,neg,neu
0,2024-11-11,0.120000,0.026500,0.853000
1,2024-11-12,0.076000,0.063667,0.860333
2,2024-11-13,0.109250,0.106750,0.783750
3,2024-11-14,0.070000,0.023000,0.907000
4,2024-11-15,0.131000,0.105000,0.764000
5,2024-11-17,0.118000,0.009000,0.872000
6,2024-11-18,0.094000,0.043500,0.862500
7,2024-11-19,0.042500,0.045500,0.912500
8,2024-11-20,0.108000,0.014000,0.877000
9,2024-11-21,0.134000,0.033000,0.833000


In [73]:
average_sentiments = postdf.groupby('Date')[['pos', 'neg', 'neu']].mean().reset_index()
average_sentiments.columns = ['Date', 'average_pos', 'average_neg', 'average_neu']
average_sentiments

,Date,average_pos,average_neg,average_neu
0,2024-11-11,0.120000,0.026500,0.853000
1,2024-11-12,0.076000,0.063667,0.860333
2,2024-11-13,0.109250,0.106750,0.783750
3,2024-11-14,0.070000,0.023000,0.907000
4,2024-11-15,0.131000,0.105000,0.764000
5,2024-11-17,0.118000,0.009000,0.872000
6,2024-11-18,0.094000,0.043500,0.862500
7,2024-11-19,0.042500,0.045500,0.912500
8,2024-11-20,0.108000,0.014000,0.877000
9,2024-11-21,0.134000,0.033000,0.833000


In [74]:
def categorize_sentiment(row):
    if row['pos'] > row['neg'] and row['pos'] > row['neu']:
        return 'positive'
    elif row['neg'] > row['pos'] and row['neg'] > row['neu']:
        return 'negative'
    else:
        return 'neutral'

In [75]:
postdf['sentiment_category'] = postdf.apply(categorize_sentiment, axis=1)
postdf

,neg,neu,pos,Date,sentiment_category
126,0.033,0.809,0.158,2024-11-11,neutral
125,0.020,0.897,0.082,2024-11-11,neutral
29,0.144,0.738,0.118,2024-11-12,neutral
325,0.047,0.843,0.110,2024-11-12,neutral
324,0.000,1.000,0.000,2024-11-12,neutral
124,0.076,0.875,0.049,2024-11-13,neutral
532,0.045,0.765,0.190,2024-11-13,neutral
323,0.125,0.783,0.091,2024-11-13,neutral
123,0.181,0.712,0.107,2024-11-13,neutral
28,0.023,0.907,0.070,2024-11-14,neutral


In [76]:
# Calculate daily sentiment ratios
sentiment_ratios = (
    postdf.groupby(['Date', 'sentiment_category'])
    .size()
    .unstack(fill_value=0)
    .reset_index()
)
sentiment_ratios

sentiment_category,Date,neutral
0,2024-11-11,2
1,2024-11-12,3
2,2024-11-13,4
3,2024-11-14,1
4,2024-11-15,1
5,2024-11-17,1
6,2024-11-18,2
7,2024-11-19,2
8,2024-11-20,1
9,2024-11-21,1


In [77]:
if 'negative' not in sentiment_ratios.columns:
    sentiment_ratios['negative'] = 0
if 'positive' not in sentiment_ratios.columns:
    sentiment_ratios['positive'] = 0

sentiment_ratios

sentiment_category,Date,neutral,negative,positive
0,2024-11-11,2,0,0
1,2024-11-12,3,0,0
2,2024-11-13,4,0,0
3,2024-11-14,1,0,0
4,2024-11-15,1,0,0
5,2024-11-17,1,0,0
6,2024-11-18,2,0,0
7,2024-11-19,2,0,0
8,2024-11-20,1,0,0
9,2024-11-21,1,0,0


In [78]:
sentiment_ratios['total_posts'] = sentiment_ratios[['positive', 'negative', 'neutral']].sum(axis=1)
sentiment_ratios['positive_ratio'] = sentiment_ratios['positive'] / sentiment_ratios['total_posts']
sentiment_ratios['negative_ratio'] = sentiment_ratios['negative'] / sentiment_ratios['total_posts']
sentiment_ratios['neutral_ratio'] = sentiment_ratios['neutral'] / sentiment_ratios['total_posts']
sentiment_ratios

sentiment_category,Date,neutral,negative,positive,total_posts,positive_ratio,negative_ratio,neutral_ratio
0,2024-11-11,2,0,0,2,0.0,0.0,1.0
1,2024-11-12,3,0,0,3,0.0,0.0,1.0
2,2024-11-13,4,0,0,4,0.0,0.0,1.0
3,2024-11-14,1,0,0,1,0.0,0.0,1.0
4,2024-11-15,1,0,0,1,0.0,0.0,1.0
5,2024-11-17,1,0,0,1,0.0,0.0,1.0
6,2024-11-18,2,0,0,2,0.0,0.0,1.0
7,2024-11-19,2,0,0,2,0.0,0.0,1.0
8,2024-11-20,1,0,0,1,0.0,0.0,1.0
9,2024-11-21,1,0,0,1,0.0,0.0,1.0


In [79]:
# Merge the average sentiments and sentiment ratios back into the original DataFrame
combined_sentiments = pd.merge(average_sentiments, sentiment_ratios, on='Date', how='inner')


combined_sentiments.drop(columns=['positive', 'negative', 'neutral', 'total_posts'], inplace=True)
combined_sentiments

,Date,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio
0,2024-11-11,0.120000,0.026500,0.853000,0.0,0.0,1.0
1,2024-11-12,0.076000,0.063667,0.860333,0.0,0.0,1.0
2,2024-11-13,0.109250,0.106750,0.783750,0.0,0.0,1.0
3,2024-11-14,0.070000,0.023000,0.907000,0.0,0.0,1.0
4,2024-11-15,0.131000,0.105000,0.764000,0.0,0.0,1.0
5,2024-11-17,0.118000,0.009000,0.872000,0.0,0.0,1.0
6,2024-11-18,0.094000,0.043500,0.862500,0.0,0.0,1.0
7,2024-11-19,0.042500,0.045500,0.912500,0.0,0.0,1.0
8,2024-11-20,0.108000,0.014000,0.877000,0.0,0.0,1.0
9,2024-11-21,0.134000,0.033000,0.833000,0.0,0.0,1.0


In [91]:
price20days

,Date,Close,High,Low,Open,Volume
2,2024-11-01,222.910004,225.350006,220.270004,220.970001,65276700
3,2024-11-04,222.009995,222.789993,219.710007,220.990005,44944500
4,2024-11-05,223.449997,223.949997,221.139999,221.800003,28111300
5,2024-11-06,222.720001,226.070007,221.190002,222.610001,54561100
6,2024-11-07,227.479996,227.880005,224.570007,224.630005,42137700
7,2024-11-08,226.960007,228.660004,226.410004,227.169998,38328800
8,2024-11-11,224.229996,225.699997,221.500000,225.000000,42005600
9,2024-11-12,224.229996,225.589996,223.360001,224.550003,40398300
10,2024-11-13,225.119995,226.649994,222.759995,224.009995,48566200
11,2024-11-14,228.220001,228.869995,225.000000,225.020004,44923900


In [97]:
#Merging and creating
merged_df = pd.merge(price20days,combined_sentiments, on='Date', how='left',)
merged_df.fillna(0.333333,inplace=True)

merged_df

,Date,Close,High,Low,Open,Volume,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio
0,2024-10-31,225.910004,229.830002,225.369995,229.339996,64370100,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
1,2024-11-01,222.910004,225.350006,220.270004,220.970001,65276700,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
2,2024-11-04,222.009995,222.789993,219.710007,220.990005,44944500,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
3,2024-11-05,223.449997,223.949997,221.139999,221.800003,28111300,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
4,2024-11-06,222.720001,226.070007,221.190002,222.610001,54561100,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
5,2024-11-07,227.479996,227.880005,224.570007,224.630005,42137700,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
6,2024-11-08,226.960007,228.660004,226.410004,227.169998,38328800,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
7,2024-11-11,224.229996,225.699997,221.500000,225.000000,42005600,0.120000,0.026500,0.853000,0.000000,0.000000,1.000000
8,2024-11-12,224.229996,225.589996,223.360001,224.550003,40398300,0.076000,0.063667,0.860333,0.000000,0.000000,1.000000
9,2024-11-13,225.119995,226.649994,222.759995,224.009995,48566200,0.109250,0.106750,0.783750,0.000000,0.000000,1.000000


In [98]:
merged_df['LogReturn_Close'] = np.log(merged_df['Close'] / merged_df['Close'].shift(1))
merged_df['LogReturn_Volume'] = np.log(merged_df['Volume'] / merged_df['Volume'].shift(1))
merged_df=merged_df.dropna()
merged_df= merged_df.drop(columns=['Close','Volume','Open','High','Low'])
merged_df

,Date,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio,LogReturn_Close,LogReturn_Volume
1,2024-11-01,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,-0.013369,0.013986
2,2024-11-04,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,-0.004046,-0.373207
3,2024-11-05,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.006465,-0.469257
4,2024-11-06,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,-0.003272,0.663150
5,2024-11-07,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,0.021147,-0.258378
6,2024-11-08,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333,-0.002288,-0.094741
7,2024-11-11,0.120000,0.026500,0.853000,0.000000,0.000000,1.000000,-0.012102,0.091601
8,2024-11-12,0.076000,0.063667,0.860333,0.000000,0.000000,1.000000,0.000000,-0.039015
9,2024-11-13,0.109250,0.106750,0.783750,0.000000,0.000000,1.000000,0.003961,0.184140
10,2024-11-14,0.070000,0.023000,0.907000,0.000000,0.000000,1.000000,0.013677,-0.077958


In [101]:
scaler=joblib.load('neural_network/dataset_prep/all_avg_ratio/avg_ratio_scaler.joblib')

In [102]:
feature_cols = [
    'LogReturn_Close', 'LogReturn_Volume', 
    'average_pos', 'average_neg', 'average_neu', 
    'positive_ratio', 'negative_ratio', 'neutral_ratio'
]

In [103]:
merged_df[feature_cols] = scaler.transform(merged_df[feature_cols])
merged_df

,Date,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio,LogReturn_Close,LogReturn_Volume
1,2024-11-01,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.349785,0.039554
2,2024-11-04,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.120522,-1.036754
3,2024-11-05,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,0.137958,-1.303751
4,2024-11-06,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.101502,1.844082
5,2024-11-07,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,0.499001,-0.717557
6,2024-11-08,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.077309,-0.262683
7,2024-11-11,0.456342,-0.426093,-0.024981,-0.226435,-0.198479,0.221398,-0.318626,0.255307
8,2024-11-12,-0.333585,0.248084,0.050390,-0.226435,-0.198479,0.221398,-0.021032,-0.107777
9,2024-11-13,0.263349,1.029585,-0.736720,-0.226435,-0.198479,0.221398,0.076381,0.512544
10,2024-11-14,-0.441303,-0.489580,0.530022,-0.226435,-0.198479,0.221398,0.315293,-0.216029


In [105]:
X=[]
group = merged_df.sort_values(by='Date').reset_index(drop=True)
group


,Date,average_pos,average_neg,average_neu,positive_ratio,negative_ratio,neutral_ratio,LogReturn_Close,LogReturn_Volume
0,2024-11-01,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.349785,0.039554
1,2024-11-04,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.120522,-1.036754
2,2024-11-05,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,0.137958,-1.303751
3,2024-11-06,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.101502,1.844082
4,2024-11-07,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,0.499001,-0.717557
5,2024-11-08,4.286288,5.139638,-5.366027,5.492880,5.864961,-5.900912,-0.077309,-0.262683
6,2024-11-11,0.456342,-0.426093,-0.024981,-0.226435,-0.198479,0.221398,-0.318626,0.255307
7,2024-11-12,-0.333585,0.248084,0.050390,-0.226435,-0.198479,0.221398,-0.021032,-0.107777
8,2024-11-13,0.263349,1.029585,-0.736720,-0.226435,-0.198479,0.221398,0.076381,0.512544
9,2024-11-14,-0.441303,-0.489580,0.530022,-0.226435,-0.198479,0.221398,0.315293,-0.216029


In [107]:

input_sequence = group[feature_cols].iloc[0: 20].values
X.append(input_sequence)
X

[array([[-0.34978515,  0.03955402,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [-0.12052229, -1.03675399,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [ 0.13795789, -1.30375126,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [-0.10150232,  1.84408197,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [ 0.49900118, -0.71755699,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [-0.07730936, -0.26268291,  4.28628831,  5.13963777, -5.36602717,
          5.4928799 ,  5.86496105, -5.90091203],
        [-0.31862634,  0.25530741,  0.45634218, -0.42609281, -0.024981  ,
         -0.22643517, -0.19847938,  0.22139825],
        [-0.02103223, -0.10777719, -0.33358544,  0.24808387,  0.05038971,
         -0.22643517, -0.19847938,  0.22139825],
        [ 0.0763

In [108]:
X = np.array(X)
print(f"Shape of X: {X.shape}")

Shape of X: (1, 20, 8)


In [110]:
model=tf.keras.models.load_model('neural_network/models/avg_ratio_lstm_larger_model.keras')

I0000 00:00:1733072601.635533    2351 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7368 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:29:00.0, compute capability: 8.6
/home/aunno/repos/stock-vision-Neural-Network/env/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 30 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [114]:
y=model(X).numpy()[0][0]
y

np.float16(0.0982)

In [117]:
placeholder_data=np.zeros((1,len(feature_cols)))
placeholder_data[0,feature_cols.index('LogReturn_Close')]=y
placeholder_data

array([[0.09820557, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ]])

In [118]:
inversed_data=scaler.inverse_transform(placeholder_data)
inversed_y=inversed_data[0,feature_cols.index('LogReturn_Close')]
inversed_y

np.float64(0.004848750660431358)

In [119]:
price20days['Close'].iloc[-1]

np.float64(237.3300018310547)

In [120]:
np.exp(inversed_y)

np.float64(1.0048605248742988)

In [121]:
predicted_price = price20days['Close'].iloc[-1] * np.exp(inversed_y)
predicted_price

np.float64(238.4835502083719)